TRABAJO PRÁCTICO 4 - NLP - Bot QA

Construir QA Bot basado en el ejemplo del traductor pero con un dataset QA.

Tener en cuenta las siguientes recomendaciones:

1. MAX_VOCAB_SIZE = 8000
2. max_length ~ 10
3. Embeddings 300 Fasttext
4. n_units = 128
5. LSTM Dropout 0.2
6. Epochs 30~50

Preguntas interesantes:
1. Do you read?
2. Do you have any pet?
3. Where are you from?

In [1]:
import re

import numpy as np
import pandas as pd

import tensorflow as tf
from keras.preprocessing.text import one_hot
from tensorflow.keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM, SimpleRNN
from keras.models import Model
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input

In [2]:
# Descargar la carpeta de dataset
import os
import gdown
if os.access('data_volunteers.json', os.F_OK) is False:
    url = 'https://drive.google.com/uc?id=1awUxYwImF84MIT5-jCaYAPe2QwSgS1hN&export=download'
    output = 'data_volunteers.json'
    gdown.download(url, output, quiet=False)
else:
    print("El dataset ya se encuentra descargado")

Downloading...
From: https://drive.google.com/uc?id=1awUxYwImF84MIT5-jCaYAPe2QwSgS1hN&export=download
To: /content/data_volunteers.json
100%|██████████| 2.58M/2.58M [00:00<00:00, 164MB/s]


In [3]:
# dataset_file
import json

text_file = "data_volunteers.json"
with open(text_file) as f:
    data = json.load(f) # la variable data será un diccionario

In [4]:
# Observar los campos disponibles en cada linea del dataset
data[0].keys()

dict_keys(['dialog', 'start_time', 'end_time', 'bot_profile', 'user_profile', 'eval_score', 'profile_match', 'participant1_id', 'participant2_id'])

In [7]:
chat_in = []
chat_out = []

input_sentences = []
output_sentences = []
output_sentences_inputs = []
max_len = 30

def clean_text(txt):
    txt = txt.lower()
    txt.replace("\'d", " had")
    txt.replace("\'s", " is")
    txt.replace("\'m", " am")
    txt.replace("don't", "do not")
    txt = re.sub(r'\W+', ' ', txt)

    return txt

for line in data:
    for i in range(len(line['dialog'])-1):
        # vamos separando el texto en "preguntas" (chat_in)
        # y "respuestas" (chat_out)
        chat_in = clean_text(line['dialog'][i]['text'])
        chat_out = clean_text(line['dialog'][i+1]['text'])

        if len(chat_in) >= max_len or len(chat_out) >= max_len:
            continue

        input_sentence, output = chat_in, chat_out

        # output sentence (decoder_output) tiene <eos>
        output_sentence = output + ' <eos>'
        # output sentence input (decoder_input) tiene <sos>
        output_sentence_input = '<sos> ' + output

        input_sentences.append(input_sentence)
        output_sentences.append(output_sentence)
        output_sentences_inputs.append(output_sentence_input)

print("Cantidad de rows utilizadas:", len(input_sentences))

Cantidad de rows utilizadas: 6033


In [8]:
input_sentences[1], output_sentences[1], output_sentences_inputs[1]

('hi how are you ', 'not bad and you  <eos>', '<sos> not bad and you ')

In [9]:
MAX_VOCAB_SIZE = 8000

In [17]:
# Tokenizar las oraciones de entrada
tokenizer_inputs = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer_inputs.fit_on_texts(input_sentences)
input_sequences = tokenizer_inputs.texts_to_sequences(input_sentences)

# Longitud máxima de las secuencias de entrada
max_input_len = 10

# Crear word2idx para las entradas
word2idx_inputs = tokenizer_inputs.word_index
num_words_input = len(word2idx_inputs) + 1

# Tokenizar las oraciones de salida
tokenizer_outputs = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer_outputs.fit_on_texts(output_sentences + output_sentences_inputs)
output_sequences = tokenizer_outputs.texts_to_sequences(output_sentences)
output_sequences_inputs = tokenizer_outputs.texts_to_sequences(output_sentences_inputs)

# Longitud máxima de las secuencias de salida
max_output_len = 10

# Añadir tokens especiales a las oraciones de salida durante la tokenización
tokenizer_outputs = Tokenizer(filters='')
tokenizer_outputs.fit_on_texts(output_sentences + output_sentences_inputs)
output_sequences = tokenizer_outputs.texts_to_sequences(output_sentences)
output_sequences_inputs = tokenizer_outputs.texts_to_sequences(output_sentences_inputs)

# Crear word2idx para las salidas
word2idx_outputs = tokenizer_outputs.word_index
word2idx_outputs['<sos>'] = tokenizer_outputs.word_index.get('<sos>', len(word2idx_outputs) + 1)
word2idx_outputs['<eos>'] = tokenizer_outputs.word_index.get('<eos>', len(word2idx_outputs) + 1)
num_words_output = min(len(word2idx_outputs) + 1, MAX_VOCAB_SIZE)

# Padding de las secuencias
encoder_input_sequences = pad_sequences(input_sequences, maxlen=max_input_len)
decoder_input_sequences = pad_sequences(output_sequences_inputs, maxlen=max_output_len)
decoder_output_sequences = pad_sequences(output_sequences, maxlen=max_output_len, padding='post')

In [18]:
print("Cantidad de rows del dataset:", len(input_sequences))
print("encoder_input_sequences shape:", encoder_input_sequences.shape)
print("decoder_input_sequences shape:", decoder_input_sequences.shape)


Cantidad de rows del dataset: 6033
encoder_input_sequences shape: (6033, 10)
decoder_input_sequences shape: (6033, 10)


In [19]:
from keras.utils import to_categorical
decoder_targets = to_categorical(decoder_output_sequences, num_classes=num_words_output)
decoder_targets.shape

(6033, 10, 1807)

In [35]:
encoder_input_sequences.shape, decoder_input_sequences.shape, decoder_output_sequences.shape

((6033, 10), (6033, 10), (6033, 10, 1, 1, 1))

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
ruta_archivo = '/content/drive/My Drive/NLP CEIA/glove_300d.txt'
embedding_dim = 300

# Cargar embeddings preentrenados (GloVe de 300 dimensiones)
embeddings_index = {}
with open(ruta_archivo, 'r', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [21]:
# Crear matriz de embedding para entradas
embedding_matrix_input = np.zeros((num_words_input, embedding_dim))
for word, i in word2idx_inputs.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_input[i] = embedding_vector

# Crear matriz de embedding para salidas
embedding_matrix_output = np.zeros((num_words_output, embedding_dim))
for word, i in word2idx_outputs.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix_output[i] = embedding_vector

In [29]:
# Crear el modelo encoder
encoder_inputs = Input(shape=(10,))
encoder_embedding = Embedding(num_words_input,embedding_dim , weights=[embedding_matrix_input], trainable=False)(encoder_inputs)
encoder_lstm = LSTM(128, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Crear el modelo decoder
decoder_inputs = Input(shape=(10,))
decoder_embedding = Embedding(num_words_output, embedding_dim, weights=[embedding_matrix_output], trainable=False)(decoder_inputs)
decoder_lstm = LSTM(128, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Definir el modelo
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

In [30]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 10)]         0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, 10, 300)      540000      ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, 10, 300)      542100      ['input_4[0][0]']                
                                                                                            

In [38]:
# Convertir las secuencias de salida en una forma compatible para entrenamiento
decoder_output_sequences = np.squeeze(decoder_output_sequences)
decoder_output_sequences.shape
# Entrenar el modelo (Lo entrené dos veces con 30 epochs)
#model_fit=model.fit([encoder_input_sequences, decoder_input_sequences], decoder_targets, batch_size=32, epochs=30, validation_split=0.2)

(6033, 10)

In [32]:
# Verificar si hay GPU disponible. NO TENGO GPU DISPONIBLE Y ME GASTÓ LAS UI :'(
gpu_available = tf.config.list_physical_devices('GPU')
if gpu_available:
    print(f"GPU disponible: {gpu_available}")
else:
    print("No hay GPU disponible.")

No hay GPU disponible.


In [33]:
print("Forma de encoder_input:", encoder_inputs.shape)  # Debe ser (n_samples, max_input_len)
print("Forma de decoder_input:", decoder_inputs.shape)  # Debe ser (n_samples, max_output_len)
print("Forma de decoder_output:", decoder_outputs.shape)  # Debe ser (n_samples, 10)


Forma de encoder_input: (None, 10)
Forma de decoder_input: (None, 10)
Forma de decoder_output: (None, 10, 1807)


In [39]:
# Modelo encoder para inferencia
encoder_model_inf = model(encoder_inputs, encoder_states)

# Modelo decoder para inferencia
decoder_state_input_h = Input(shape=(128,))
decoder_state_input_c = Input(shape=(128,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model_inf = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_sequence(input_seq):
    states_value = encoder_model_inf.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model_inf.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer_outputs.index_word.get(sampled_token_index, '')

        if sampled_word == '<eos>' or len(decoded_sentence) > max_output_len:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        states_value = [h, c]

    return decoded_sentence


def preprocess_sentence(sentence):
    # Limpiar la oración de entrada
    sentence = clean_text(sentence)
    # Convertir la oración en una secuencia de índices
    sequence = tokenizer_inputs.texts_to_sequences([sentence])
    # Hacer padding a la secuencia para que tenga la longitud máxima de entrada
    padded_sequence = pad_sequences(sequence, maxlen=max_input_len)
    return padded_sequence

def decode_sequence(input_seq):
    # Obtener los estados internos del encoder
    states_value = encoder_model_inf.predict(input_seq)

    # Generar una secuencia vacía de longitud 1 con el índice del token de inicio
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']

    decoded_sentence = ''
    stop_condition = False
    while not stop_condition:
        # Obtener las predicciones del decoder
        output_tokens, h, c = decoder_model_inf.predict([target_seq] + states_value)

        # Obtener el índice del token con mayor probabilidad
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer_outputs.index_word.get(sampled_token_index, '')

        if sampled_word == '<eos>' or len(decoded_sentence.split()) > max_output_len:
            stop_condition = True
        else:
            decoded_sentence += ' ' + sampled_word

        # Actualizar la secuencia objetivo (target_seq)
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # Actualizar los estados del decoder
        states_value = [h, c]

    return decoded_sentence.strip()

AssertionError: Exception encountered when calling layer "model_1" (type Functional).

Could not compute output KerasTensor(type_spec=TensorSpec(shape=(None, 10, 1807), dtype=tf.float32, name=None), name='dense_1/Softmax:0', description="created by layer 'dense_1'")

Call arguments received by layer "model_1" (type Functional):
  • inputs=tf.Tensor(shape=(None, 10), dtype=float32)
  • training=['tf.Tensor(shape=(None, 128), dtype=float32)', 'tf.Tensor(shape=(None, 128), dtype=float32)']
  • mask=None

In [ ]:
# Probar el modelo
input_sentence = "Where are you from"
preprocessed_sentence = preprocess_sentence(input_sentence)
decoded_sentence = decode_sequence(preprocessed_sentence)
print(f'Q: {input_sentence}')
print(f'A: {decoded_sentence}')

In [13]:
import pickle